In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import h5py

/Users/abulbasar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import requests
url = "https://raw.githubusercontent.com/abulbasar/machine-learning/master/Utils.py"
exec(requests.get(url).text)

In [3]:
def load_mnist_csv(path = "/data/MNIST/", one_hot = False, shape = None):
    df_train = pd.read_csv(path + "mnist_train.csv", header=None)
    df_test = pd.read_csv(path + "mnist_test.csv", header=None)
    
    X_train = df_train.iloc[:, 1:].values/255
    X_test = df_test.iloc[:, 1:].values/255
    y_train = df_train.iloc[:, 0].values
    y_test = df_test.iloc[:, 0].values
    
    if shape == "2D":
        X_train = X_train.reshape(-1, 28, 28)
        X_test = X_test.reshape(-1, 28, 28)
    
    if one_hot:
        eye = np.eye(len(np.unique(y_train)))
        y_train, y_test = eye[y_train], eye[y_test]
        
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_mnist_csv(shape = "2D")


In [5]:
tf.reset_default_graph()
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

max_iters = 5000

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                          logits=logits)
loss = tf.reduce_mean(xentropy)
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()


n_epochs = 100
batch_size = 150

batchable = Batchable(X_train, y_train, batch_size=32)

with tf.Session() as sess:
    init.run()
    for i in range(max_iters):
        X_batch, y_batch = batchable.next()
        sess.run(opt, feed_dict={X: X_batch, y: y_batch})
        
        if i % (max_iters//20) == 0 or i == max_iters -1:
            acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(i, "Train accuracy:", acc_train, "test accuracy: ", acc_test)

0 Train accuracy: 0.15625 test accuracy:  0.1229
250 Train accuracy: 0.90625 test accuracy:  0.8759
500 Train accuracy: 0.96875 test accuracy:  0.8799
750 Train accuracy: 0.9375 test accuracy:  0.9013
1000 Train accuracy: 0.78125 test accuracy:  0.9217
1250 Train accuracy: 0.9375 test accuracy:  0.9245
1500 Train accuracy: 0.96875 test accuracy:  0.9238
1750 Train accuracy: 1.0 test accuracy:  0.919
2000 Train accuracy: 0.90625 test accuracy:  0.9325
2250 Train accuracy: 0.90625 test accuracy:  0.9322
2500 Train accuracy: 0.96875 test accuracy:  0.9274
2750 Train accuracy: 1.0 test accuracy:  0.9407
3000 Train accuracy: 0.9375 test accuracy:  0.9514
3250 Train accuracy: 0.96875 test accuracy:  0.9439
3500 Train accuracy: 0.96875 test accuracy:  0.9385
3750 Train accuracy: 0.9375 test accuracy:  0.9407
4000 Train accuracy: 0.96875 test accuracy:  0.9541
4250 Train accuracy: 0.96875 test accuracy:  0.9475
4500 Train accuracy: 0.9375 test accuracy:  0.9371
4750 Train accuracy: 0.90625 tes